In [6]:
import pandas as pd
from sqlalchemy import create_engine, text

# Create and engine for the database
engine = create_engine('sqlite:///Maji_Ndogo_farm_survey_small.db')

In [7]:
# Check if the database is created and tables are present
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM sqlite_master WHERE type='table';"))
    for row in result:
        print(row)

('table', 'geographic_features', 'geographic_features', 2, 'CREATE TABLE geographic_features (\n\t"Field_ID" BIGINT, \n\t"Elevation" FLOAT, \n\t"Latitude" FLOAT, \n\t"Longitude" FLOAT, \n\t"Location" TEXT, \n\t"Slope" FLOAT\n)')
('table', 'weather_features', 'weather_features', 81, 'CREATE TABLE weather_features (\n\t"Field_ID" BIGINT, \n\t"Rainfall" FLOAT, \n\t"Min_temperature_C" FLOAT, \n\t"Max_temperature_C" FLOAT, \n\t"Ave_temps" FLOAT\n)')
('table', 'soil_and_crop_features', 'soil_and_crop_features', 138, 'CREATE TABLE soil_and_crop_features (\n\t"Field_ID" BIGINT, \n\t"Soil_fertility" FLOAT, \n\t"Soil_type" TEXT, \n\t"pH" FLOAT\n)')
('table', 'farm_management_features', 'farm_management_features', 182, 'CREATE TABLE farm_management_features (\n\t"Field_ID" BIGINT, \n\t"Pollution_level" FLOAT, \n\t"Plot_size" FLOAT, \n\t"Crop_type" FLOAT, \n\t"Annual_yield" TEXT, \n\t"Standard_yield" FLOAT\n)')


In [11]:
# Combine all the tables in a single table using the Field_ID column

sql_query = """
    SELECT
        g.Field_ID,
        g.Elevation,
        g.Latitude,
        g.Longitude,
        g.Location,
        g.Slope,
        w.Rainfall,
        w.Min_temperature_C,
        w.Max_temperature_C,
        w.Ave_temps,
        s.Soil_fertility,
        s.soil_type,
        s.ph,
        f.Pollution_level,
        f.Plot_size,
        f.Crop_type,
        f.Annual_yield,
        f.Standard_yield
    FROM
        geographic_features g
    JOIN
        weather_features w ON g.Field_ID = w.Field_ID
    JOIN
        soil_and_crop_features s ON g.Field_ID = s.Field_ID
    JOIN
            farm_management_features f ON g.Field_ID = f.Field_ID;
    """

In [ ]:
# Create a pandas dataframe using the SQL query
engine = create_engine('sqlite:///Maji_Ndogo_farm_survey_small.db')
MD_agric_df = pd.read_sql(text(sql_query), engine)

# Display the first few rows of the combined dataframe
MD_agric_df.head()

,Field_ID,Elevation,Latitude,Longitude,Location,Slope,Rainfall,Min_temperature_C,Max_temperature_C,Ave_temps,Soil_fertility,Soil_type,pH,Pollution_level,Plot_size,Crop_type,Annual_yield,Standard_yield
0,40734,786.05580,-7.389911,-7.556202,Rural_Akatsi,14.795113,1125.2,-3.1,33.1,15.00,0.62,Sandy,6.169393,0.085267,1.3,0.751354,cassava,0.577964
1,30629,674.33410,-7.736849,-1.051539,Rural_Sokoto,11.374611,1450.7,-3.9,30.6,13.35,0.64,Volcanic,5.676648,0.399684,2.2,1.069865,cassava,0.486302
2,39924,826.53390,-9.926616,0.115156,Rural_Sokoto,11.339692,2208.9,-1.8,28.4,13.30,0.69,Volcanic,5.331993,0.358029,3.4,2.208801,tea,0.649647
3,5754,574.94617,-2.420131,-6.592215,Rural_Kilimani,7.109855,328.8,-5.8,32.2,13.20,0.54,Loamy,5.328150,0.286687,2.4,1.277635,cassava,0.532348
4,14146,886.35300,-3.055434,-7.952609,Rural_Kilimani,55.007656,785.2,-2.5,31.0,14.25,0.72,Sandy,5.721234,0.043190,1.5,0.832614,wheat,0.555076


### Data Cleanup

In [ ]:

# Swap the 'Crop_type' and 'Annual_yield' columns
MD_agric_df = MD_agric_df.rename(columns={'Crop_type': 'Annual_yield', 'Annual_yield': 'Crop_type'})

MD_agric_df.head()

,Field_ID,Elevation,Latitude,Longitude,Location,Slope,Rainfall,Min_temperature_C,Max_temperature_C,Ave_temps,Soil_fertility,Soil_type,pH,Pollution_level,Plot_size,Annual_yield,Crop_type,Standard_yield
0,40734,786.05580,-7.389911,-7.556202,Rural_Akatsi,14.795113,1125.2,-3.1,33.1,15.00,0.62,Sandy,6.169393,0.085267,1.3,0.751354,cassava,0.577964
1,30629,674.33410,-7.736849,-1.051539,Rural_Sokoto,11.374611,1450.7,-3.9,30.6,13.35,0.64,Volcanic,5.676648,0.399684,2.2,1.069865,cassava,0.486302
2,39924,826.53390,-9.926616,0.115156,Rural_Sokoto,11.339692,2208.9,-1.8,28.4,13.30,0.69,Volcanic,5.331993,0.358029,3.4,2.208801,tea,0.649647
3,5754,574.94617,-2.420131,-6.592215,Rural_Kilimani,7.109855,328.8,-5.8,32.2,13.20,0.54,Loamy,5.328150,0.286687,2.4,1.277635,cassava,0.532348
4,14146,886.35300,-3.055434,-7.952609,Rural_Kilimani,55.007656,785.2,-2.5,31.0,14.25,0.72,Sandy,5.721234,0.043190,1.5,0.832614,wheat,0.555076


In [14]:
# Correct crop names
MD_agric_df['Crop_type'].unique()

array(['cassava', 'tea', 'wheat', 'potato', 'banana', 'coffee', 'rice',
       'maize', 'wheat ', 'tea ', 'cassaval', 'wheatn', 'cassava ',
       'teaa'], dtype=object)

In [15]:
def correct_crop_names(crop_name):
    corrections = {
        'cassava ': 'cassava',
        'wheat ': 'wheat',
        'tea ': 'tea',
        'cassaval': 'cassava',
        'wheatn': 'wheat',
        'teaa': 'tea',
    }
    return corrections.get(crop_name, crop_name)

MD_agric_df['Crop_type'] = MD_agric_df['Crop_type'].apply(correct_crop_names)

MD_agric_df['Crop_type'].unique()

array(['cassava', 'tea', 'wheat', 'potato', 'banana', 'coffee', 'rice',
       'maize'], dtype=object)

In [ ]:
# Change negative values in the Elevation column to positive values
MD_agric_df['Elevation'] = MD_agric_df['Elevation'].abs() 

count    5654.000000
mean      637.790709
std       173.573934
min        35.910797
25%       525.611413
50%       663.058585
75%       764.299550
max      1122.252100
Name: Elevation, dtype: float64

dtype('float64')